In [1]:
import pyvo as vo
import pandas as pd

# Define the Gaia DR3 source_id you're interested in
gaia_source_ids = [1234567890123456789, 9876543210987654321]  # Replace with your list of Gaia source_ids
service_gaia = vo.dal.TAPService("http://gea.esac.esa.int/tap-server/tap")

In [2]:
gaia_query = """
        SELECT
        gs.source_id, gs.original_ext_source_id
        FROM gaiadr3.panstarrs1_best_neighbour as gs
        WHERE gs.source_id in {}
        """.format(tuple(gaia_source_ids))
gaia_result = service_gaia.search(gaia_query).to_table()

DALFormatError: E19: None:2:0: E19: File does not appear to be a VOTABLE

In [ ]:

# Simplify the initial query to avoid complex joins and test the service
# We'll query Gaia and PANSTARRS separately and cross-match locally

# Step 1: Query Gaia to get positions for the source_ids
gaia_query = f"""
SELECT source_id, ra, dec
FROM gaiadr3.gaia_source
WHERE source_id IN {tuple(gaia_source_ids)}
"""
print(gaia_query)
gaia_result = service_gaia.search(gaia_query)

# Convert Gaia result to a pandas DataFrame
df_gaia = gaia_result.to_table().to_pandas()
print("Gaia Data:")
print(df_gaia)

In [ ]:
# Step 2: Query PANSTARRS for objects near the Gaia positions
# Use a small search radius, e.g., 1 arcsecond (0.00027778 degrees)
panstarrs_query = f"""
SELECT objID, raMean, decMean, g_psfMag, r_psfMag, i_psfMag, z_psfMag, y_psfMag, 
       g_psfMagErr, r_psfMagErr, i_psfMagErr, z_psfMagErr, y_psfMagErr,
       gFlags, rFlags, iFlags, zFlags, yFlags, nDetections
FROM panstarrs_dr1.meanobject
WHERE 1=CONTAINS(
    POINT('ICRS', raMean, decMean),
    CIRCLE('ICRS', {df_gaia['ra'][0]}, {df_gaia['dec'][0]}, 0.00027778)
)
AND g_psfMagErr < 0.05 AND r_psfMagErr < 0.05 AND i_psfMagErr < 0.05
AND z_psfMagErr < 0.05 AND y_psfMagErr < 0.05
AND (gFlags & 0x0000000F) = 0 AND (rFlags & 0x0000000F) = 0 
AND (iFlags & 0x0000000F) = 0 AND (zFlags & 0x0000000F) = 0 
AND (yFlags & 0x0000000F) = 0 AND nDetections >= 2
"""
service_panstarrs = vo.dal.TAPService("https://archive.stsci.edu/panstarrs/tapservice")
panstarrs_result = service_panstarrs.search(panstarrs_query)

# Convert PANSTARRS result to a pandas DataFrame
df_panstarrs = panstarrs_result.to_table().to_pandas()
print("PANSTARRS Data:")
print(df_panstarrs)

# Step 3: (Optional) Cross-match Gaia and PANSTARRS results locally if needed
# Here, it's done by matching positions (RA, DEC) within a small tolerance
cross_matched = pd.merge(df_gaia, df_panstarrs, left_on=['ra', 'dec'], right_on=['raMean', 'decMean'], how='inner')
print("Cross-Matched Data:")
print(cross_matched)